In [1]:
import pandas as pd
import numpy as np


In [2]:
# 1. Price Returns
def calc_price_return(df):
    return df['Price'].pct_change()

# 2. Log Returns
def calc_log_return(df):
    return np.log(df['Price'] / df['Price'].shift(1))

# 3. Volatility (default 20 days, adjust accordingly)
def calc_volatility(df, window=20):
    returns = df['Price'].pct_change()
    return returns.rolling(window=window).std()

# 4. Moving Average (default 5 days, adjust accordingly)
def calc_MA(df, window=5):
    return df['Price'].rolling(window=window).mean()

# 5 RSI (default 14-day, adjust accordingly)
def calc_rsi(df, window=14):
    delta = df['Price'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean() # Avg gain
    loss = (delta.where(delta < 0, 0)).rolling(window=window).mean() # Avg loss
    RS = gain / loss # RS = Avg gain / Avg loss
    return 100 - (100 / (1 + RS))

# 6 Moving Average Convergence Divergence (MACD)
def calc_macd(df):
    ema_12 = df['Price'].ewm(span=12, adjust=False).mean()
    ema_26 = df['Price'].ewm(span=26, adjust=False).mean()
    return ema_12 - ema_26

# 7 Beta: Measure volatility of stock relative to market
# mkt_returns is typically returns of a mkt index such as SP500
def calc_beta(df, mkt_returns):
    covariance = np.cov(df['Price'].pct_change().dropna(), mkt_returns)[0][0]
    mkt_variance = np.nanvar(mkt_returns)
    return covariance / mkt_variance   
    
# 8 Sharpe ratio: Performance / Risk (1Y US T-bills usually considered)
def calc_sharpe_ratio(df, risk_free_rate=0.01):
    returns = df['Price'].pct_change()
    excess_returns = returns - risk_free_rate / 252
    sharpe_ratio = excess_returns.mean() / excess_returns.std()
    return sharpe_ratio

# 9 Maximum Drawdown: Largest drop from peak to trough
def calc_max_draw_down(df):
    cumulative_returns = (1 + df['Price'].pct_change().cumprod()) # daily time-period
    peak = cumulative_returns.cummax() # [1., 1.1, 1.05, 1.2] -> [1., 1.1, 1.1, 1.2]
    drawdown = (cumulative_returns - peak) / peak # Measures percentage drop from peak at any point
    return drawdown.min() # return most negative value (worst drop)

# 10 Price-Earning ratio
def calc_pe_ratio(df):
    return df['Price'] / df['EPS']

In [3]:
amd_df = pd.read_csv("../data/stocks-AMD-Jan2020-Dec2024.csv")
boe_df = pd.read_csv("../data/stocks-BOE-Jan2020-Dec2024.csv")
snp_df = pd.read_csv("../data/stocks-SNPETF-Jan2020-Dec2024.csv")
tsla_df = pd.read_csv("../data/stocks-TSLA-Jan2020-Dec2024.csv")



In [4]:
print(amd_df.shape)
print(boe_df.shape)
print(snp_df.shape)
print(tsla_df.shape)

(1238, 7)
(1238, 7)
(1268, 7)
(1238, 7)


In [5]:
snp_df.dropna(inplace=True)

In [6]:
print(amd_df.isna().sum())
print(boe_df.isna().sum())
print(snp_df.isna().sum())
print(tsla_df.isna().sum())

Date        0
Price       0
Open        0
High        0
Low         0
Vol.        0
Change %    0
dtype: int64
Date        0
Price       0
Open        0
High        0
Low         0
Vol.        0
Change %    0
dtype: int64
Date        0
Price       0
Open        0
High        0
Low         0
Vol.        0
Change %    0
dtype: int64
Date        0
Price       0
Open        0
High        0
Low         0
Vol.        0
Change %    0
dtype: int64


### Dtype Cleaning for Stocks


In [7]:
amd_df.head(1)

,Date,Price,Open,High,Low,Vol.,Change %
0,12/02/2024,142.06,137.91,142.82,137.8,33.10M,3.56%


In [8]:
amd_df['Date'] = pd.to_datetime(amd_df['Date'])
boe_df['Date'] = pd.to_datetime(boe_df['Date'])
snp_df['Date'] = pd.to_datetime(snp_df['Date'])
tsla_df['Date'] = pd.to_datetime(tsla_df['Date'])

amd_df['Year'] = amd_df['Date'].dt.year
boe_df['Year'] = boe_df['Date'].dt.year
snp_df['Year'] = snp_df['Date'].dt.year
tsla_df['Year'] = tsla_df['Date'].dt.year


In [9]:
AMD_eps_data = {
    'Year': [2020, 2021, 2022, 2023, 2024],
    'EPS': [2.06, 2.57, 0.84, 0.53, 0.7]
}

TSLA_eps_data = {
    'Year': [2020, 2021, 2022, 2023, 2024],
    'EPS': [0.21, 1.63, 3.62, 4.3, 3.64]
}

GSPC_eps_data = {
    'Year': [2020, 2021, 2022, 2023, 2024],
    'EPS': [9.66, 12.51, 10.20, 8.23,11.33]
}

BOE_eps_data = {
    'Year': [2020, 2021, 2022, 2023, 2024],
    'EPS': [-20.88, -7.15, -8.3, -3.67, -13]
}

AMD_eps_df = pd.DataFrame(AMD_eps_data)
TSLA_eps_df = pd.DataFrame(TSLA_eps_data)
GSPC_eps_df = pd.DataFrame(GSPC_eps_data)
BOE_eps_df = pd.DataFrame(BOE_eps_data)

amd_df = amd_df.merge(AMD_eps_df, on='Year', how='left')
tsla_df = tsla_df.merge(TSLA_eps_df, on='Year', how='left')
snp_df = snp_df.merge(GSPC_eps_df, on='Year', how='left')
boe_df = boe_df.merge(BOE_eps_df, on='Year', how='left')



In [10]:
amd_df.head(5)

,Date,Price,Open,High,Low,Vol.,Change %,Year,EPS
0,2024-12-02,142.06,137.91,142.82,137.80,33.10M,3.56%,2024,0.7
1,2024-11-29,137.18,136.24,138.59,135.78,16.09M,0.69%,2024,0.7
2,2024-11-27,136.24,137.20,137.94,132.96,30.18M,-1.07%,2024,0.7
3,2024-11-26,137.72,142.55,142.80,136.62,32.09M,-2.42%,2024,0.7
4,2024-11-25,141.13,140.49,142.35,139.05,30.92M,2.01%,2024,0.7


In [11]:
def convert_vol(vol):
    if isinstance(vol, str):  # Check if the value is a string
        vol = vol.strip()
        if 'M' in vol:
            return float(vol.replace('M', '')) * 1_000_000
        elif 'K' in vol:
            return float(vol.replace('K', '')) * 1_000
        elif 'B' in vol:
            return float(vol.replace('B', '')) * 1_000_000_000
    return float(vol)  # Return the value as float if it's not a string with 'M', 'K', or 'B'

# Apply the conversion to the 'Vol.' column
amd_df['Vol.'] = amd_df['Vol.'].apply(convert_vol)
tsla_df['Vol.'] = tsla_df['Vol.'].apply(convert_vol)
snp_df['Vol.'] = snp_df['Vol.'].apply(convert_vol)
boe_df['Vol.'] = boe_df['Vol.'].apply(convert_vol)

In [12]:
amd_df['Change %'] = amd_df['Change %'].replace({'%': ''}, regex=True).astype(float) / 100
tsla_df['Change %'] = tsla_df['Change %'].replace({'%': ''}, regex=True).astype(float) / 100
snp_df['Change %'] = snp_df['Change %'].replace({'%': ''}, regex=True).astype(float) / 100
boe_df['Change %'] = boe_df['Change %'].replace({'%': ''}, regex=True).astype(float) / 100

In [13]:
amd_df['Ticker'] = 'AMD'
tsla_df['Ticker'] = 'TSLA'
snp_df['Ticker'] = 'GSPC'
boe_df['Ticker'] = 'BOE'

### Feature Engineering for Stocks

    - Price Returns
    - Log Returns
    - Volatility
    - Moving Average
    - Relative Strength Index (RSI)
    - Moving Average Convergence Divergence
    - Beta
    - Sharpe Ratio
    - Maximum Drawdown
    - Price-to-Earnings ratio
    - Dividend Yield
    - Volume Moving Average (VMA)
    - Exponential Moving Average


In [15]:
def generateFeatures(combined_df):
    combined_df['Price_Return'] = calc_price_return(combined_df)
    combined_df['Log_Return'] = calc_log_return(combined_df)
    combined_df['Volatility'] = calc_volatility(combined_df)
    combined_df['Moving_Average'] = calc_MA(combined_df)
    combined_df['RSI'] = calc_rsi(combined_df)
    combined_df['MACD'] = calc_macd(combined_df)
    mkt_returns = combined_df[combined_df['Ticker'] == 'GSPC']['Price'].pct_change()
    mkt_returns = np.tile(mkt_returns, 4)
    # combined_df['Beta'] = calc_beta(combined_df, mkt_returns[1:]) # assume 2% lowest risk returns
    combined_df['Sharpe_Ratio'] = calc_sharpe_ratio(combined_df)
    combined_df['Max_Drawdown'] = calc_max_draw_down(combined_df)

df_list = [amd_df, tsla_df, snp_df, boe_df]
for df in df_list:
    generateFeatures(df)

In [18]:
for df in df_list:
    df.dropna(inplace=True)

In [19]:
amd_df.isna().sum()

Date              0
Price             0
Open              0
High              0
Low               0
Vol.              0
Change %          0
Year              0
EPS               0
Ticker            0
Price_Return      0
Log_Return        0
Volatility        0
Moving_Average    0
RSI               0
MACD              0
Sharpe_Ratio      0
Max_Drawdown      0
dtype: int64

In [20]:
amd_df.shape

(1218, 18)

In [21]:
combined_df = pd.concat(df_list, ignore_index=True)


In [22]:
combined_df.shape

(4872, 18)

In [23]:
combined_df.to_csv("../data/stocks-COMBINED-Jan2020-Dec2024.csv")